In [1]:
import logging
import warnings
import datalink
import sys

warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s | %(message)s',
                    level=logging.DEBUG,
                    stream=sys.stdout)
log = logging.getLogger(__name__)

In [2]:
# Datalink supports the logging module.
datalink.test_output()

# Datalink logging can be disabled with
# logging.getLogger('datalink').propagate = False

INFO | logging from datalink


## Generate user classes with the `link_factory`

A data store class is generated by the link_factory method.

The user can then create instances of this class in a similar way to 

creating `namedtuple` objects from the `collections` module.

In [3]:
MyStore = datalink.link_factory(name='MyStore', db_path='/tmp/test.db', table_name='data',
                                data_fields={'a': None, 'b': [2,4], 'c': 'a string'})

In [4]:
d = MyStore()

DEBUG | Creating database: /tmp/test.db
INFO | - db created at path: /tmp/test.db


# Accessing data fields.

The internal data store is exposed via the data property.


In [5]:
d.data

{'a': None, 'b': [2, 4], 'c': 'a string'}

Data is accessible through properties corresponding to the field name.

In [6]:
print(d.a)
d.b
d.c

None


[2, 4]

'a string'

## Data store auto saving.

The first time a property in the store is changed from its default value,

the store is persisted in the SQL database linked to the class.

In [7]:
d.a = 12
d.data

DEBUG | Creating new database entry.


{'a': 12, 'b': [2, 4], 'c': 'a string'}

In [8]:
# The update method can be used to set multiple field values 
# with only one update pushed to the SQL db.
d.update(a=14, c='a new string')
d.data

DEBUG | Updating existing database entry for uuid 56da7a95-9847-4960-9d40-47d8910f9016.


{'a': 14, 'b': [2, 4], 'c': 'a new string'}

By default, the class is designed to handle unique data, accessible

via a uuid generated with `uuid.uuid4`.

In [9]:
my_uuid = d.uuid
my_uuid

'56da7a95-9847-4960-9d40-47d8910f9016'

## Loading data into a new instance of MyClass.

To load previously saved data, pass the uuid as the `datalink_uuid`

keyword arg to the instance declaration.

In [10]:
del(d)
d2 = MyStore(datalink_uuid=my_uuid)

DEBUG | Loading data corresponding to uuid f56da7a95-9847-4960-9d40-47d8910f9016


In [11]:
d2.data

{'a': 14, 'b': [2, 4], 'c': 'a new string'}

In [12]:
# Link to the entry in the SQL db is maintained.
d2.a = 16
d2.data

DEBUG | Updating existing database entry for uuid 56da7a95-9847-4960-9d40-47d8910f9016.


{'a': 16, 'b': [2, 4], 'c': 'a new string'}